In [ ]:
import os, sys

sys.path.append(os.path.dirname(os.path.dirname(os.path.abspath("__file__"))))
from nbafuns import *
from nba_api.stats.endpoints import leaguedashteamstats, leaguedashplayerstats
from great_tables import GT, md, html

fig_DIR = "../figs/team_ratings/"
img_DIR_T = "../data/images/teams/"

df_teams = pd.read_csv("../data/NBA_teams_colors_logos.csv")
df_teams = df_teams.rename(columns={"nameTeam": "TEAM_NAME"})

In [ ]:
stats = leaguedashteamstats.LeagueDashTeamStats(
    measure_type_detailed_defense="Advanced",
    per_mode_detailed="PerGame",
    # date_from_nullable="10-23-2023",
    date_to_nullable="12-25-2023",
)
df11 = stats.get_data_frames()[0]
stats = leaguedashteamstats.LeagueDashTeamStats(
    measure_type_detailed_defense="Advanced",
    per_mode_detailed="PerGame",
    date_from_nullable="12-26-2023",
    date_to_nullable="02-18-2024",
)
df21 = stats.get_data_frames()[0]
stats = leaguedashteamstats.LeagueDashTeamStats(
    measure_type_detailed_defense="Advanced",
    per_mode_detailed="PerGame",
    date_from_nullable="02-18-2024",
    # date_to_nullable=",
)
df31 = stats.get_data_frames()[0]

In [ ]:
cols = ['TEAM_NAME', 'GP', 'W_PCT',
        'OFF_RATING', 'DEF_RATING','NET_RATING', 
        'W_PCT_RANK', 'OFF_RATING_RANK', 'DEF_RATING_RANK', 'NET_RATING_RANK', ]

In [ ]:
df1 = df11[cols]
df2 = df21[cols]
df3 = df31[cols]
df3.columns  = df3.columns + '_3'
df3 = df3.rename(columns={"TEAM_NAME_3":"TEAM_NAME"})

In [ ]:
df41 = pd.merge(df1,df2,on=["TEAM_NAME"],suffixes=["_1","_2"])
df4 =  pd.merge(df41,df3,on=["TEAM_NAME"],suffixes=["","_3"])
df4 = pd.merge(df4,df_teams,on=["TEAM_NAME"])
df4["image"]= img_DIR_T + df4["TEAM_NAME"] + ".png"

In [ ]:
df4[["GP_1","GP_2","GP_3"]].mean()

In [ ]:
p = (ggplot(df4)
 + geom_point(aes(x=1, y='NET_RATING_RANK_1', color='colorsTeam'), size=2.5, alpha=.7)
 + geom_point(aes(x=2, y='NET_RATING_RANK_2', color='colorsTeam'), size=2.5, alpha=.7)
 + geom_point(aes(x=3, y='NET_RATING_RANK_3', color='colorsTeam'), size=2.5, alpha=.7)
 + geom_image(aes(x=1, y='NET_RATING_RANK_1', image='image'),size=0.08)
 + geom_image(aes(x=2, y='NET_RATING_RANK_2', image='image'),size=0.08)
 + geom_image(aes(x=3, y='NET_RATING_RANK_3', image='image'),size=0.08)
 + geom_segment(aes(x=1, y='NET_RATING_RANK_1', xend=2, yend='NET_RATING_RANK_2', color='colorsTeam'), alpha=.7, size=1)
 + geom_segment(aes(x=2, y='NET_RATING_RANK_2', xend=3, yend='NET_RATING_RANK_3', color='colorsTeam'), alpha=.7, size=1)
 + annotate('text', x=1, y=0, label='Till Christmas', size=14)
 + annotate('text', x=2, y=0, label='Christmas - ASB', size=14)
 + annotate('text', x=3, y=0, label='ASB-', size=14)
 + lims(x=(0.8, 3.2))
 + scale_y_reverse(breaks=np.arange(1,31,1))
 + scale_color_identity(aesthetics=["color"])
 + theme_idv
 + theme(figure_size=(8, 15))
 + theme(
     axis_title_x = element_blank(),
     axis_text_x = element_blank(),
     plot_title=element_text(face="bold", size=20),
 )
 + labs(
        title=f"Net Rating Rankings for 3 Sections of the NBA Season",
        subtitle = "1. Start - Christmas | 2. Christmas - All Star Break | 3. All Star Break - Now",
        caption="@sradjoker | source: nba.com/stats",
        y=f"Net Rating Rank",
 )
)
# p.save("../figs/team_leaders/LA_aDEF_mvmt.png", dpi=300, verbose = False)
# p

In [ ]:
df4["T"] = df4["TEAM_NAME"]
df5 = df4[[
    'T','TEAM_NAME','NET_RATING_RANK_1','NET_RATING_RANK_2','NET_RATING_RANK_3'
]].sort_values("NET_RATING_RANK_3").reset_index(drop=True).reset_index()
df5["index"] +=1


In [ ]:
t = (
    GT(df5)
    .tab_header(
        title=md("Net Rating Rankings for 3 Sections of the NBA Season"),
        # subtitle="Daily measurements in New York City (May 1-10, 1973)"
    )
    .tab_source_note(source_note="@sradjoker | source: nba.com/stats")
    .cols_label(
        index = "#",
        TEAM_NAME = "Team",
        NET_RATING_RANK_1 = "Till Christmas",
        NET_RATING_RANK_2 = "Christmas - ASB",
        NET_RATING_RANK_3 = "ASB - Now",
    )
    .cols_align(align="center")
    .cols_align(align="left", columns="TEAM_NAME")
    .data_color(
        columns=['NET_RATING_RANK_1','NET_RATING_RANK_2','NET_RATING_RANK_3'],
        palette="RdYlGn",
        reverse="True"
    )
    .fmt_image(
        columns="T",
        path = img_DIR_T,
        file_pattern="{}.png"
    )
    .tab_options(
        heading_title_font_size="150%",
        heading_subtitle_font_size="110%",
        # heading_title_font_weight='bold',
        table_background_color="floralwhite",
        column_labels_font_size="105%",
        column_labels_font_weight='bold',
        row_group_font_weight='bold',
        row_group_background_color="#E5E1D8",
        table_font_size=10,
        table_font_names="Consolas", 
        data_row_padding = "3px",
        table_margin_left = 1,
        table_margin_right = 1,
    )
    # .save(fig_DIR + "net_season_segment.png")
)
t.save("net_season_segment.png",scale=3,expand=2,web_driver="edge",window_size=(6000,6000))
# t

In [ ]:
df4["T"] = df4["TEAM_NAME"]
df5 = df4[[
    'T','TEAM_NAME','OFF_RATING_RANK_1','OFF_RATING_RANK_2','OFF_RATING_RANK_3'
]].sort_values("OFF_RATING_RANK_3").reset_index(drop=True).reset_index()
df5["index"] +=1


In [ ]:
t = (
    GT(df5)
    .tab_header(
        title=md("Off Rating Rankings for 3 Sections of the NBA Season"),
        # subtitle="Daily measurements in New York City (May 1-10, 1973)"
    )
    .tab_source_note(source_note="@sradjoker | source: nba.com/stats")
    .cols_label(
        index = "#",
        TEAM_NAME = "Team",
        OFF_RATING_RANK_1 = "Till Christmas",
        OFF_RATING_RANK_2 = "Christmas - ASB",
        OFF_RATING_RANK_3 = "ASB - Now",
    )
    .cols_align(align="center")
    .cols_align(align="left", columns="TEAM_NAME")
    .data_color(
        columns=['OFF_RATING_RANK_1','OFF_RATING_RANK_2','OFF_RATING_RANK_3'],
        palette="RdYlGn",
        reverse="True"
    )
    .fmt_image(
        columns="T",
        path = img_DIR_T,
        file_pattern="{}.png"
    )
    .tab_options(
        heading_title_font_size="150%",
        heading_subtitle_font_size="110%",
        # heading_title_font_weight='bold',
        table_background_color="floralwhite",
        column_labels_font_size="105%",
        column_labels_font_weight='bold',
        row_group_font_weight='bold',
        row_group_background_color="#E5E1D8",
        table_font_size=10,
        table_font_names="Consolas", 
        data_row_padding = "3px",
        table_margin_left = 1,
        table_margin_right = 1,
    )
    # .save(fig_DIR + "net_season_segment.png")
)
t.save("off_season_segment.png",scale=3,expand=2,web_driver="edge",window_size=(6000,6000))
# t

In [ ]:
df4["T"] = df4["TEAM_NAME"]
df5 = df4[[
    'T','TEAM_NAME','DEF_RATING_RANK_1','DEF_RATING_RANK_2','DEF_RATING_RANK_3'
]].sort_values("DEF_RATING_RANK_3").reset_index(drop=True).reset_index()
df5["index"] +=1


In [ ]:
t = (
    GT(df5)
    .tab_header(
        title=md("Def Rating Rankings for 3 Sections of the NBA Season"),
        # subtitle="Daily measurements in New York City (May 1-10, 1973)"
    )
    .tab_source_note(source_note="@sradjoker | source: nba.com/stats")
    .cols_label(
        index = "#",
        TEAM_NAME = "Team",
        DEF_RATING_RANK_1 = "Till Christmas",
        DEF_RATING_RANK_2 = "Christmas - ASB",
        DEF_RATING_RANK_3 = "ASB - Now",
    )
    .cols_align(align="center")
    .cols_align(align="left", columns="TEAM_NAME")
    .data_color(
        columns=['DEF_RATING_RANK_1','DEF_RATING_RANK_2','DEF_RATING_RANK_3'],
        palette="RdYlGn",
        reverse="True"
    )
    .fmt_image(
        columns="T",
        path = img_DIR_T,
        file_pattern="{}.png"
    )
    .tab_options(
        heading_title_font_size="150%",
        heading_subtitle_font_size="110%",
        # heading_title_font_weight='bold',
        table_background_color="floralwhite",
        column_labels_font_size="105%",
        column_labels_font_weight='bold',
        row_group_font_weight='bold',
        row_group_background_color="#E5E1D8",
        table_font_size=10,
        table_font_names="Consolas", 
        data_row_padding = "3px",
        table_margin_left = 1,
        table_margin_right = 1,
    )
    # .save(fig_DIR + "net_season_segment.png")
)
t.save("def_season_segment.png",scale=3,expand=2,web_driver="edge",window_size=(6000,6000))
# t